<a href="https://colab.research.google.com/github/ishancoderr/Neural-networks/blob/main/Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

**Importing the dataSet**

In [15]:
url="https://raw.githubusercontent.com/ishancoderr/Neural-networks/main/movies.dat"
url2="https://raw.githubusercontent.com/ishancoderr/Neural-networks/main/users.dat"
url3="https://raw.githubusercontent.com/ishancoderr/Neural-networks/main/ratings.dat"
movies=pd.read_csv(url,sep='::',header=None,engine='python',encoding='latin-1')
users=pd.read_csv(url2,sep='::',header=None,engine='python',encoding='latin-1')
ratings=pd.read_csv(url3,sep='::',header=None,engine='python',encoding='latin-1')

In [21]:
url6="https://raw.githubusercontent.com/ishancoderr/Neural-networks/main/u1.base"
training_set = pd.read_csv(url6, delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
url5="https://raw.githubusercontent.com/ishancoderr/Neural-networks/main/u1.test"
test_set =pd.read_csv(url5,delimiter='\t')
test_set =np.array(test_set,dtype='int')